# Imports

In [1]:
import os
from natsort import natsorted
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
import math
from math import log10
import numpy as np
# nltk.download('punkt')

# First part

1. Read (.txt) files [Test set of 10 files was provided for this coursework]
2. Apply tokenization
3. Apply stemming

In [2]:
def apply_tokenization_and_stemming(text):
    # initializing the stemmer. in our case it is the PorterStemmer
    stemmer = PorterStemmer()

    # tokenizing words
    tokenization = word_tokenize(text)

    # declearing the list of stemmed words
    stemmed_words = []

    # looping over the tokenzied words and applying stemming then appending each stemmed word to 'stemmed_words'
    for word in tokenization:
        stemmed_words.append(stemmer.stem(word))

    # returning a list of words that are tokenized and stemmed
    return stemmed_words


In [3]:
document_collection = natsorted(os.listdir('DocumentCollection'))

list_of_terms = []

for doc in document_collection:
    with open(f'DocumentCollection/{doc}', 'r') as f:
            document = f.read()
    list_of_terms.append(apply_tokenization_and_stemming(document))

# Second part

## (1) Creating the positional index

In [4]:
# Starting second part (1) positional index

positional_index = {}

for doc_id, terms in enumerate(list_of_terms, start=1):
    for position, term in enumerate(terms, start=1):
        # if the term doesn't exist in our positional index we add it
        if term not in positional_index:
              positional_index[term] = {
                   'doc_count': 1,
                   'docs': {doc_id: [position]}
              }
        # if the term already exists we do the following
        else:
            # first, add to the count of that term
            positional_index[term]['doc_count'] += 1
            # if the doc_id is new, this means we have a new document that contain the same word
            # we add a the doc_id to our positional index along with the position of that word in that doc_id
            if doc_id not in positional_index[term]['docs']:
                positional_index[term]['docs'][doc_id] = [position]
            # if the doc_id already exists, then we add the position of that term within that doc_id
            else:
                positional_index[term]['docs'][doc_id].append(position)


In [5]:
for doc_id, terms in enumerate(list_of_terms, start=1):
     print(doc_id, terms)
        
positional_index

1 ['antoni', 'brutu', 'caeser', 'cleopatra', 'merci', 'worser']
2 ['antoni', 'brutu', 'caeser', 'calpurnia']
3 ['merci', 'worser']
4 ['brutu', 'caeser', 'merci', 'worser']
5 ['caeser', 'merci', 'worser']
6 ['antoni', 'caeser', 'merci']
7 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where']
8 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where']
9 ['angel', 'fool', 'in', 'rush', 'to', 'tread', 'where']
10 ['fool', 'fear', 'in', 'rush', 'to', 'tread', 'where', 'fool']


{'antoni': {'doc_count': 3, 'docs': {1: [1], 2: [1], 6: [1]}},
 'brutu': {'doc_count': 3, 'docs': {1: [2], 2: [2], 4: [1]}},
 'caeser': {'doc_count': 5, 'docs': {1: [3], 2: [3], 4: [2], 5: [1], 6: [2]}},
 'cleopatra': {'doc_count': 1, 'docs': {1: [4]}},
 'merci': {'doc_count': 5, 'docs': {1: [5], 3: [1], 4: [3], 5: [2], 6: [3]}},
 'worser': {'doc_count': 4, 'docs': {1: [6], 3: [2], 4: [4], 5: [3]}},
 'calpurnia': {'doc_count': 1, 'docs': {2: [4]}},
 'angel': {'doc_count': 3, 'docs': {7: [1], 8: [1], 9: [1]}},
 'fool': {'doc_count': 5, 'docs': {7: [2], 8: [2], 9: [2], 10: [1, 8]}},
 'fear': {'doc_count': 3, 'docs': {7: [3], 8: [3], 10: [2]}},
 'in': {'doc_count': 4, 'docs': {7: [4], 8: [4], 9: [3], 10: [3]}},
 'rush': {'doc_count': 4, 'docs': {7: [5], 8: [5], 9: [4], 10: [4]}},
 'to': {'doc_count': 4, 'docs': {7: [6], 8: [6], 9: [5], 10: [5]}},
 'tread': {'doc_count': 4, 'docs': {7: [7], 8: [7], 9: [6], 10: [6]}},
 'where': {'doc_count': 4, 'docs': {7: [8], 8: [8], 9: [7], 10: [7]}}}

## (2) Computing Term Frequency (tf)

Term frequency is how many times did the term appear in a document

Our positional index now contain a dictionary in the following format

{'term': {'doc_count': No. of docs containing term, 'docs': {doc_id: [positions]} } }

Based on our positional index, term frequency can be calculated by getting the length of the
[positions] array in doc_id. 

note: doc_id key holds positions that a term appeard. Multiple positions within the same doc_id is the term frequency for that doc_id

In [6]:
term_frequency = {}

for term, info in positional_index.items():
    term_frequency[term] = {}
    for doc_id, positions in info['docs'].items():
         term_frequency[term][f"Doc{doc_id}"] = len(positions)

In [7]:
# converting our tf table to a pandas dataframe so we can display it
tf_table = pd.DataFrame(term_frequency).transpose().fillna(0).astype(int)
# sorting columns
tf_table = tf_table[natsorted(tf_table.columns)]

## tf

In [8]:
from tabulate import tabulate
print("DataFrame:")
print(tabulate(tf_table, headers='keys', tablefmt='fancy_grid'))

DataFrame:
╒═══════════╤════════╤════════╤════════╤════════╤════════╤════════╤════════╤════════╤════════╤═════════╕
│           │   Doc1 │   Doc2 │   Doc3 │   Doc4 │   Doc5 │   Doc6 │   Doc7 │   Doc8 │   Doc9 │   Doc10 │
╞═══════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪═════════╡
│ antoni    │      1 │      1 │      0 │      0 │      0 │      1 │      0 │      0 │      0 │       0 │
├───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┤
│ brutu     │      1 │      1 │      0 │      1 │      0 │      0 │      0 │      0 │      0 │       0 │
├───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┤
│ caeser    │      1 │      1 │      0 │      1 │      1 │      1 │      0 │      0 │      0 │       0 │
├───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┤
│ cleopatra │      1 │      0 │      0 │    

In [9]:
w_tf = tf_table.copy()

for c_name, c_value in w_tf.items():
    # print(f"Column: {c_value}")
    for index, value in c_value.items():
        if w_tf.loc[index, c_name] != 0:
            w_tf.loc[index, c_name] = (1 + log10(w_tf.loc[index, c_name]))

## w_tf(1 + log(tf))

In [10]:
w_tf

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8,Doc9,Doc10
antoni,1,1,0,0,0,1,0,0,0,0.00000
brutu,1,1,0,1,0,0,0,0,0,0.00000
caeser,1,1,0,1,1,1,0,0,0,0.00000
cleopatra,1,0,0,0,0,0,0,0,0,0.00000
merci,1,0,1,1,1,1,0,0,0,0.00000
worser,1,0,1,1,1,0,0,0,0,0.00000
calpurnia,0,1,0,0,0,0,0,0,0,0.00000
angel,0,0,0,0,0,0,1,1,1,0.00000
fool,0,0,0,0,0,0,1,1,1,1.30103
fear,0,0,0,0,0,0,1,1,0,1.00000


## (3) Computing IDF

IDF measures how rare a term is by dividing the number of documents (N) by the how many documents did the term appear in - the document frequency (df)

The equation is: IDF = log_10(N/df)

In [11]:
df_idf_table = pd.DataFrame(columns=['df', 'idf'])

N = len(document_collection)

for i, term in enumerate(positional_index):
    df_idf_table.loc[i, 'df'] = tf_table.loc[term].sum(axis=0)
    df_idf_table.loc[i, 'idf'] = log10(N/df_idf_table.loc[i, 'df'])
    
df_idf_table.index = tf_table.index
df_idf_table

,df,idf
antoni,3,0.522879
brutu,3,0.522879
caeser,5,0.30103
cleopatra,1,1.0
merci,5,0.30103
worser,4,0.39794
calpurnia,1,1.0
angel,3,0.522879
fool,5,0.30103
fear,3,0.522879


## (4) Computing TF-IDF

In [12]:
tf_idf_table = tf_table.copy()

for i in range(1, tf_table.shape[1] + 1):
    tf_idf_table[f'Doc{i}'] *= df_idf_table['idf'].values
    
    
    
tf_idf_table

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8,Doc9,Doc10
antoni,0.522879,0.522879,0.0,0.0,0.0,0.522879,0.0,0.0,0.0,0.0
brutu,0.522879,0.522879,0.0,0.522879,0.0,0.0,0.0,0.0,0.0,0.0
caeser,0.30103,0.30103,0.0,0.30103,0.30103,0.30103,0.0,0.0,0.0,0.0
cleopatra,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
merci,0.30103,0.0,0.30103,0.30103,0.30103,0.30103,0.0,0.0,0.0,0.0
worser,0.39794,0.0,0.39794,0.39794,0.39794,0.0,0.0,0.0,0.0,0.0
calpurnia,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
angel,0.0,0.0,0.0,0.0,0.0,0.0,0.522879,0.522879,0.522879,0.0
fool,0.0,0.0,0.0,0.0,0.0,0.0,0.30103,0.30103,0.30103,0.60206
fear,0.0,0.0,0.0,0.0,0.0,0.0,0.522879,0.522879,0.0,0.522879


## Normalized length of docs

In [13]:
doc_length_index = [f"{col} length" for col in tf_idf_table.columns]

doc_length = pd.DataFrame(columns=["Euclidean length"], index=doc_length_index)

for c_name, c_value in tf_idf_table.items():
    doc_length.loc[f"{c_name} length", "Euclidean length"] = math.sqrt((c_value ** 2).sum())

doc_length

,Euclidean length
Doc1 length,1.373462
Doc2 length,1.279618
Doc3 length,0.498974
Doc4 length,0.782941
Doc5 length,0.582747
Doc6 length,0.67427
Doc7 length,1.195493
Doc8 length,1.195493
Doc9 length,1.075083
Doc10 length,1.194847


## Normalized tf.idf

In [14]:
normalized_tf_idf = tf_idf_table.copy()

for c_name, c_value in normalized_tf_idf.items():
    doc_length_value = doc_length.loc[f"{c_name} length", "Euclidean length"]
    normalized_tf_idf[c_name] = normalized_tf_idf[c_name] / doc_length_value

normalized_tf_idf

,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8,Doc9,Doc10
antoni,0.380701,0.408621,0.0,0.0,0.0,0.775474,0.0,0.0,0.0,0.0
brutu,0.380701,0.408621,0.0,0.667839,0.0,0.0,0.0,0.0,0.0,0.0
caeser,0.219176,0.23525,0.0,0.384486,0.51657,0.446453,0.0,0.0,0.0,0.0
cleopatra,0.728087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
merci,0.219176,0.0,0.603298,0.384486,0.51657,0.446453,0.0,0.0,0.0,0.0
worser,0.289735,0.0,0.797516,0.508263,0.682869,0.0,0.0,0.0,0.0,0.0
calpurnia,0.0,0.781483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
angel,0.0,0.0,0.0,0.0,0.0,0.0,0.437375,0.437375,0.486361,0.0
fool,0.0,0.0,0.0,0.0,0.0,0.0,0.251804,0.251804,0.280006,0.50388
fear,0.0,0.0,0.0,0.0,0.0,0.0,0.437375,0.437375,0.0,0.437611


## (5) Phrase query

we should apply the same preprocessing to the query as we did to our collection. In other words we should apply tokenization and stemming. This works on exact postions of the phrase query

In [15]:
def intersect(postings):
    # check if postings list is empty
    if len(postings) == 0:
        return 0
    else:
        intersection_set = set(postings[0])

        for posting in postings[1:]:
            intersection_set = intersection_set.intersection(posting)
            

        return natsorted(intersection_set)

def matched_docs(query):
    query_terms = query
    postings_list = []
    for term in query_terms:
        if term in positional_index:
            postings_list.append(list(positional_index[term]['docs'].keys()))
        else:
            print("No documents found for selected query terms")
            return 0
            
    # ordering the postings list based on df (this will help with intersection)
    ordered_postings = sorted(postings_list, key=lambda item: len(item))
    
    # print(ordered_postings)
    
    # computed matched docs with intersect function on the ordered postings
    matched_docs = intersect(ordered_postings)
    
    # after looping the query_terms if no terms are matched with positional index
    # the postings_list will be empty and return 0
    if not (bool(matched_docs)):
        return 0
        
    # else we intersect term's postings lists
    else:
        return matched_docs

In [16]:
doc_id_positions = {key: value['docs'] for key, value in positional_index.items()}

doc_id_positions

{'antoni': {1: [1], 2: [1], 6: [1]},
 'brutu': {1: [2], 2: [2], 4: [1]},
 'caeser': {1: [3], 2: [3], 4: [2], 5: [1], 6: [2]},
 'cleopatra': {1: [4]},
 'merci': {1: [5], 3: [1], 4: [3], 5: [2], 6: [3]},
 'worser': {1: [6], 3: [2], 4: [4], 5: [3]},
 'calpurnia': {2: [4]},
 'angel': {7: [1], 8: [1], 9: [1]},
 'fool': {7: [2], 8: [2], 9: [2], 10: [1, 8]},
 'fear': {7: [3], 8: [3], 10: [2]},
 'in': {7: [4], 8: [4], 9: [3], 10: [3]},
 'rush': {7: [5], 8: [5], 9: [4], 10: [4]},
 'to': {7: [6], 8: [6], 9: [5], 10: [5]},
 'tread': {7: [7], 8: [7], 9: [6], 10: [6]},
 'where': {7: [8], 8: [8], 9: [7], 10: [7]}}

### this one puts term with its repective postions

In [17]:
def process_query(query):
    stemmed_query = apply_tokenization_and_stemming(query)
    operator = ""
    switch = 0
    phrase_one = []
    phrase_two = []
    
    for term in stemmed_query:
        if switch == 0:
            phrase_one.append(term)
        elif switch == 1:
            phrase_two.append(term)

        if term == "and":
            switch = 1
            phrase_one.pop()
            operator = "and"

        elif term == "or":
            switch = 1
            phrase_one.pop()
            operator = "or"

        elif term == "not":
            switch = 1
            phrase_one.pop()
            operator = "not"
    return phrase_one, operator, phrase_two

In [18]:


def phrase_query(phrase):
    
    list_of_docs = matched_docs(phrase)
    
    # matched docs for the phrase
    if len(phrase) == 1 or len(phrase) == 0:
        return list_of_docs
        
    else:
        if list_of_docs == 0:
            return 0
        else:
            
            #print(list_of_docs)
            postings = []

            # get the positions of matched docs for each term in query
            for term in phrase:
                postings_term = []
                for i in list_of_docs:
                    # print(doc_id_positions[term][i])
                    postings_term.append(int(doc_id_positions[term][i][0]))
                postings.append(postings_term)

            #print(postings)

            # using the 666 method
            # postings will have same values for a successfully detected phrase
            for index, list_of_positions in enumerate(postings):
                for pos_idx, position in enumerate(list_of_positions):
                    postings[index][pos_idx] = position + (len(postings) - index)


            final_list = []
            for index, list_of_positions in enumerate(postings):
                for i in range(len(postings[index])):
                    if postings[index][i] == postings[index+1][i]:
                        final_list.append(list_of_docs[i])
                break


            #print(list_of_docs)
            #print(postings)


            return final_list

In [19]:
query = input("Please enter phrase query: ")

phrase_one, operator, phrase_two = process_query(query)

def boolean_operation(phrase_one, operator, phrase_two):
    matched_phrase1 = phrase_query(phrase_one)
    matched_phrase2 = phrase_query(phrase_two)
    print(matched_phrase1, operator, matched_phrase2)
    
    terms = []
    result = matched_phrase1
    
    if bool(phrase_two) == 0:
        if bool(result) != 0:
            for term in phrase_one:
                terms.append(term)

            for term in phrase_two:
                terms.append(term)
            return terms, result
        else:
            return 0
        
    if operator == "and":
        if bool(matched_phrase2) == 0:
            print("No matched docs for current phrase query")
            return 0
        else:
            result = list(set(matched_phrase1).intersection(set(matched_phrase2)))
            if bool(result) == 0:
                print("No matched docs for current phrase query")
                return 0
            else:
                for term in phrase_one:
                    terms.append(term)

                for term in phrase_two:
                    terms.append(term)
                return terms, result


    elif operator == "or":
        if bool(matched_phrase2) == 0:
            for term in phrase_one:
                terms.append(term)
            return terms, result
        else:
            for term in phrase_one:
                terms.append(term)

            for term in phrase_two:
                terms.append(term)

            for i in matched_phrase2:
                result.append(i)

            return terms, list(set(result))

    elif operator == "not":
        if bool(matched_phrase2) == 0:
            for term in phrase_one:
                terms.append(term)
            return terms, result
        else:
            for i in matched_phrase2:
                if i in result:
                    result.remove(i)

            if bool(result) == 0:
                print("No matched docs for current phrase query")
                return 0
            else:
                for term in phrase_one:
                    terms.append(term)
                return terms, result        
    

    
boolean_operation(phrase_one, operator, phrase_two)


Please enter phrase query: antony brutus
[1, 2]  0


(['antoni', 'brutu'], [1, 2])

## (6) Compute similarity between query and matched docs

similarity between a document and a query sim(d, q) is defined by - assuming each one is an array of weight = dot_product(d, q) / euclidean_length(d, q). The result is a similary score between the document and the query

In [20]:
phrase_one, operator, phrase_two = process_query(input("Please enter phrase query: "))

if bool(boolean_operation(phrase_one, operator, phrase_two)) != 0:
    query_terms, query_result = boolean_operation(phrase_one, operator, phrase_two)
else:
    query_result = 0

query_tf = {}
if bool(query_result) == 0:
    print("Not results found for query")
else:
    for term in query_terms:
        if term in query_tf:
            query_tf[term] += 1
        else:
            query_tf[term] = 1
    #print(query_tf)
    query_stat = pd.DataFrame(columns=["tf-raw", "tf(1 + log tf)", "idf", "tf*idf", "Normalized"], index=query_tf.keys())

    query_stat['tf-raw'] = query_tf.values()
    #print(query_stat)

    query_stat["tf(1 + log tf)"] = 1 + query_stat["tf-raw"].apply(log10)

    query_idf = []
    for term in query_terms:
        query_idf.append(float(df_idf_table.loc[term, "idf"]))

    query_stat["idf"] = query_idf
    
    query_stat["tf*idf"] = query_stat["tf(1 + log tf)"] *  query_stat["idf"]
    
    query_length = math.sqrt((query_stat["tf*idf"] **2).sum())
    
    print(query_length)
    
    query_stat["Normalized"] = query_stat["tf*idf"] / query_length
    
    for doc in query_result:
        query_stat[f"Doc{doc}"] = 0
    
    query_stat.loc["sum"] = np.nan
    
    for query_term in query_terms:
        for i in range(len(query_result)):
            #print(query_term, query_result[i])
            query_stat.loc[query_term, f"Doc{query_result[i]}"] = query_stat.loc[query_term, "Normalized"] * normalized_tf_idf.loc[query_term, f"Doc{query_result[i]}"]
               
    for doc in query_result:
        query_stat.loc["sum", f"Doc{doc}"] = query_stat[f"Doc{doc}"].sum()


# print(query_result)


query_stat



Please enter phrase query: antony brutus and caeser 
[1, 2] and [1, 2, 4, 5, 6]
[1, 2] and [1, 2, 4, 5, 6]
0.7983880152039714


,tf-raw,tf(1 + log tf),idf,tf*idf,Normalized,Doc1,Doc2
antoni,1.0,1.0,0.522879,0.522879,0.654918,0.249328,0.267613
brutu,1.0,1.0,0.522879,0.522879,0.654918,0.249328,0.267613
caeser,1.0,1.0,0.301030,0.301030,0.377047,0.082640,0.088700
sum,NaN,NaN,NaN,NaN,NaN,0.581296,0.623927


In [21]:
print(pd.Series(query_stat.loc["sum"].dropna().sort_values(ascending=False)))

Doc2    0.623927
Doc1    0.581296
Name: sum, dtype: float64


In [22]:
def sim_score(query_stat, doc):
    doc = f"Doc{doc}"
    if doc in query_stat.keys():
        return query_stat.loc["sum", doc]
    else:
        print(f"There is no similarity score between the query and {doc}")
        
def doc_rank(query_stat):
    rank = pd.Series(query_stat.loc["sum"].dropna().sort_values(ascending=False))
    return list(rank.index)
        
sim_score(query_stat, 1)
doc_rank(query_stat)

['Doc2', 'Doc1']